In [ ]:
import tweepy
from TweetTransformations import checkClaims, translate_to_german,  translate_to_english, deconstructTwitterQueryResponse
from config import access_token, access_token_secret, bearer_token, consumer_key, consumer_secret, API_KEY


In [ ]:
api = tweepy.Client(
    access_token=access_token,
    access_token_secret=access_token_secret,
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    wait_on_rate_limit=False)

In [ ]:
# keyword search for klimawandel 
response = api.search_recent_tweets(query="Klimawandel -is:retweet -is:reply", 
                                    expansions=["author_id",'referenced_tweets.id'],
                                    tweet_fields=["reply_settings"],
                                    user_fields=["protected"],
                                    max_results=20)

In [ ]:
response.data[0].id

In [ ]:
tweet_list = deconstructTwitterQueryResponse(response)

In [ ]:
response.data

In [ ]:
list_of_tweets = []
for tweet in response.data:
    # CORRECTED LOGIC: Check the 'reply_settings' attribute directly
    if tweet.reply_settings != 'everyone':
        print(f"Skipping Tweet {tweet.id}: Replies are limited to '{tweet.reply_settings}'.")
        continue  # Skip to the next tweet
    
    # If the check passes, you can process or add the tweet
    print(f"OK to reply to Tweet {tweet.id}")
    tweet_id = tweet.id
    original_tweet = tweet.text
    answer_dict = {
            "original_tweet" : original_tweet,
            "tweet_id": tweet_id
        }
    list_of_tweets.append(answer_dict)
    

In [ ]:
list_of_tweets

In [ ]:
tweet_list = list_of_tweets

In [ ]:
translated_tweet_list = []
for tweet in tweet_list:
    tweet_id = tweet.get("tweet_id")
    original_tweet= tweet.get("original_tweet")
    translated_original_tweet = await translate_to_english(original_tweet)
    answer_dict = {
                "original_tweet" : original_tweet,
                "tweet_id": tweet_id,
                "translated_original_tweet" : translated_original_tweet,
            }
    translated_tweet_list.append(answer_dict)

In [ ]:
translated_tweet_list

In [ ]:
list_of_factchecked_tweets = await checkClaims(tweetlist=translated_tweet_list)

In [ ]:
len(list_of_factchecked_tweets)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.llms.openrouter import OpenRouter
"""llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)"""
llm=OpenRouter(
    api_key=API_KEY,
    context_window=8128,
    model="google/gemma-3-12b-it",
    temperature=0.0
)

In [ ]:
for tweet in list_of_factchecked_tweets:
    answer_tweet = tweet.get("answer_tweet")
    tweet_id = tweet.get("tweet_id")
    translated_answer_tweet = await translate_to_german(answer_tweet)
    # check if over character limit
    if len(translated_answer_tweet) > 280:
        translated_answer_tweet = await llm.acomplete(f"Dies ist ein Tweet. Dieser ist jedoch deutlich zu lang. Kürze den tweet auf so kurz wie möglich. ab besten unter 30 Wörter. Halte dich dabei umgangssprachlich {translated_answer_tweet}")
    print(tweet_id)
    # post tweet
    try:
        api.create_tweet(text=translated_answer_tweet.text,in_reply_to_tweet_id=tweet_id)
    except tweepy.errors.Forbidden as e:
        print(f"tried to write {translated_answer_tweet} to {tweet_id}")
        print(f"error while posting: {e}")
